In [1]:
import pandas as pd
import numpy as np

df_stations = pd.read_csv('station.csv')
df_trips = pd.read_csv('trip.csv')

In [2]:
df_trips['start_time'] = pd.to_datetime(df_trips['start_time'])
df_trips['end_time'] = pd.to_datetime(df_trips['end_time'])
df_stations['station_name'] = df_stations['station_name'].apply(lambda x: x.strip())

In [3]:
#A
total_trips = len(df_trips)
#grab length of df_trip
total_trips

1569966

In [4]:
#B
municipalities = df_stations['municipality'].nunique()
#grab unique elements from stations 
municipalities

4

In [5]:
#C
station_counts = df_stations.groupby('municipality')['station_id'].nunique().reset_index()
#group by municipality, count unique station ids
station_counts.columns = ['municipality', 'station_count']
station_counts = station_counts.sort_values(by=['station_count', 'municipality'], ascending=[False, True])
#print results without index
print(station_counts.to_string(index=False))



municipality  station_count
      Boston             97
   Cambridge             28
  Somerville             12
   Brookline              5


In [6]:
#D
female_reg_users = df_trips.loc[(df_trips['user_type'] == 'Registered') & 
                               (df_trips['gender'] == 'Female')]
#grab female registered users
total_reg_users = len(df_trips.loc[df_trips['user_type'] == 'Registered'])
#count total num
female_reg_user_trips = len(female_reg_users)
#grab length of users
percent_female_reg_users = (female_reg_user_trips / total_reg_users) * 100
percent_female_reg_users

24.550994647858012

In [7]:
#E
short_trips = df_trips.loc[df_trips['duration'] < 60]
#filter by duration
total_trips = len(df_trips)
#grab length of trips in dataset
short_trip_count = len(short_trips)
#grab number of trips
percent_short_trips = (short_trip_count / total_trips) * 100
percent_short_trips = round(percent_short_trips, 2)
percent_short_trips

0.5

In [8]:
#F
#merge dataframes for start/end
longest_trip_data = (
    df_trips.merge(df_stations.rename(columns={'station_id': 'start_station_id', 'station_name': 'station_name_start'}), 
                   on='start_station_id', how='left') #grab start
            .merge(df_stations.rename(columns={'station_id': 'end_station_id', 'station_name': 'station_name_end'}), 
                   on='end_station_id', how='left') #end
                   #grab correct row
            .loc[df_trips['duration'].idxmax(), ['id', 'station_name_start', 'station_name_end', 'duration', 'bike_id']]
)
longest_trip_data

id                                  541247
station_name_start    Tremont St / West St
station_name_end      Tremont St / West St
duration                              9999
bike_id                             T01078
Name: 534848, dtype: object

In [9]:
#G
grouped_trips = df_trips.groupby('user_type')['duration'].mean()
#group trips by type, grab mean
ratio = grouped_trips['Casual'] / grouped_trips['Registered']
#calculate ratio
print(round(ratio, 2))

2.31


In [10]:
#H
roundtrips_mask = df_trips['start_station_id'] == df_trips['end_station_id']
#find trips start and end
percentage_roundtrips = roundtrips_mask.mean() * 100
#grab mean
print(round(percentage_roundtrips, 2))


4.65


In [11]:
#I
somerville_stations = set(df_stations[df_stations['municipality'] == 'Somerville']['station_id'])
#grab stations
bike_stations = df_trips.groupby('bike_id')[['start_station_id', 'end_station_id']].apply(lambda x: set(x['start_station_id']) | set(x['end_station_id']))
#grab stations from bikes
bikes_visited_somerville = bike_stations[bike_stations.apply(lambda x: somerville_stations.issubset(x))].index
#filter
print(sorted(bikes_visited_somerville))


['B00001', 'B00014', 'B00018', 'B00028', 'B00092', 'B00172', 'B00215', 'B00217', 'B00242', 'B00314', 'B00317', 'B00318', 'B00326', 'B00337', 'B00368', 'B00378', 'B00409', 'B00459', 'B00471', 'B00482', 'B00546', 'B00553', 'B00601', 'B00613', 'B01462', 'T01001', 'T01020', 'T01043', 'T01062', 'T01066', 'T01073', 'T01084', 'T01127', 'T01135', 'T01185', 'T01206', 'T01209', 'T01263', 'T01297', 'T01301', 'T01306', 'T01321', 'T01411', 'T01437']


In [12]:
#J
start_stations = df_trips.merge(df_stations, left_on='start_station_id', right_on='station_id')
#merge trips and stations
end_stations = df_trips.merge(df_stations, left_on='end_station_id', right_on='station_id')
#merge
all_stations = pd.concat([start_stations, end_stations])
#combine
bike_counts = all_stations.groupby('bike_id')['station_id'].nunique().reset_index()
#find unique
bike_counts.columns = ['bike_id', 'station_count']
#set columns
bike_counts = bike_counts.sort_values(by=['station_count', 'bike_id'], ascending=[False, True]).head(10)
bike_counts



,bike_id,station_count
17,B00001,134
469,B00456,132
565,B00553,132
926,T01214,132
366,B00353,131
283,B00270,130
331,B00318,130
384,B00371,130
451,B00438,130
495,B00482,130


In [15]:
#K

# merge stations and trips dataframes to get the start and end municipalities
df_trips = pd.merge(df_trips, df_stations[['station_id', 'municipality']], left_on='start_station_id', right_on='station_id', how='left')
df_trips = pd.merge(df_trips, df_stations[['station_id', 'municipality']], left_on='end_station_id', right_on='station_id', how='left', suffixes=('_start', '_end'))

# create a new column with a list of unique municipalities for each trip
df_trips['municipalities'] = df_trips[['municipality_start', 'municipality_end']].apply(lambda x: list(set(filter(pd.notnull, x))), axis=1)

# explode the list of municipalities so that each row represents one municipality for each trip
df_trips = df_trips.explode('municipalities')

# group by municipality and count the number of trips for each
trips_by_municipality = df_trips.groupby('municipalities').size().reset_index(name='count')

# calculate the total number of trips
total_trips = trips_by_municipality['count'].sum()

# calculate the percentage of trips for each municipality
trips_by_municipality['percentage'] = trips_by_municipality['count'] / total_trips * 100

# sort by percentage (decreasing) and then by municipality name
trips_by_municipality = trips_by_municipality.sort_values(['percentage', 'municipalities'], ascending=[False, True])

# print the output
print(trips_by_municipality[['municipalities', 'percentage']])







  municipalities  percentage
0         Boston   71.512794
2      Cambridge   22.910453
3     Somerville    3.550168
1      Brookline    2.026585


In [5]:
num = [x for x in range(1,1001) if '3' in str(x)]
num

[3,
 13,
 23,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 43,
 53,
 63,
 73,
 83,
 93,
 103,
 113,
 123,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 143,
 153,
 163,
 173,
 183,
 193,
 203,
 213,
 223,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 243,
 253,
 263,
 273,
 283,
 293,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 403,
 413,
 423,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
